# Tutorial for Reef Workflow v1.0
This is not done on the best dataset. We don't have ground truth labels for the "unlabeled" dataset and no held out test set. However, it walks you through the overall procedure. Contact paroma@stanford.edu for additional (possibly cooler) datasets.

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load Dataset
For this tutorial, we look at sample data that [Macrobase](http://macrobase.stanford.edu) uses to evaluate their anomaly detection system. Download the file `sample_labeled.csv` from [here](http://paroma.github.io/sample_labeled.csv) and place in a folder called `mobile_mb` under `reef/data/`

### Explore Data
We will take a look at the data at hand - the task is to predict which data points are outliers based on the `usage`, `latency`, `loaction`, and `version` features. 

In [2]:
import pandas as pd
df = pd.read_csv('data/mobile_mb/sample_labeled.csv', sep=',',header=0)

df

,usage,latency,location,version,inlier,outlier
0,30.770,238,CAN,v2,False,False
1,31.280,611,CAN,v2,False,False
2,31.170,768,RUS,v4,False,False
3,30.940,192,AUS,v3,False,False
4,35.360,401,UK,v3,True,False
5,39.120,531,RUS,v4,False,False
6,33.900,223,UK,v3,False,False
7,40.090,582,USA,v1,False,False
8,2.897,391,CAN,v3,False,True
9,39.030,441,CAN,v2,False,False


### Convert Data
For this tutorial, we will designate data points that are outliers as `+1` and inliers as `-1`. The rest that are neither will be part of the unlabeled dataset. 

In [3]:
#Convert categorical features to one-hot vector features
df_cat = pd.get_dummies(df)
df_cat

,usage,latency,inlier,outlier,location_AUS,location_CAN,location_RUS,location_UK,location_USA,version_v1,version_v2,version_v3,version_v4
0,30.770,238,False,False,0,1,0,0,0,0,1,0,0
1,31.280,611,False,False,0,1,0,0,0,0,1,0,0
2,31.170,768,False,False,0,0,1,0,0,0,0,0,1
3,30.940,192,False,False,1,0,0,0,0,0,0,1,0
4,35.360,401,True,False,0,0,0,1,0,0,0,1,0
5,39.120,531,False,False,0,0,1,0,0,0,0,0,1
6,33.900,223,False,False,0,0,0,1,0,0,0,1,0
7,40.090,582,False,False,0,0,0,0,1,1,0,0,0
8,2.897,391,False,True,0,1,0,0,0,0,0,1,0
9,39.030,441,False,False,0,1,0,0,0,0,1,0,0


In [4]:
#primitive matrix, ground
primitive_matrix_train = []
primitive_matrix_val = []

ground_train = []
ground_val = []

for i in range(df_cat.values.shape[0]):
    is_val = df_cat.values[i,2] or df_cat.values[i,3]
    is_outlier = df_cat.values[i,3]

    if is_val:
        primitive_matrix_val.append(df_cat.values[i,4:12])
        ground_val.append(is_outlier)
    else:
        primitive_matrix_train.append(df_cat.values[i,4:12])
        ground_train.append(is_outlier)
        
np.save('data/mobile_mb/primitive_matrix_val.npy', np.array(primitive_matrix_val).astype(float))
np.save('data/mobile_mb/ground_val.npy', 2*np.array(ground_val).astype(float)-1.)
np.save('data/mobile_mb/primitive_matrix_train.npy', np.array(primitive_matrix_train).astype(float))
np.save('data/mobile_mb/ground_train.npy', 2*np.array(ground_train).astype(float)-1.)

## REEF: Automatically Generating Heuristics to Label Training Data

### Load Data
We load in the data that we generate above. The `train_` variables are the data points we do not have ground truth labels for. The `val_` variables are data points we have ground truth labels for.

In [5]:
from data.loader import DataLoader
dataset='mobile_mb'

#replace data_folder with location where .txt files are saved
dl = DataLoader()

train_primitive_matrix, val_primitive_matrix, test_primitive_matrix, train_ground, val_ground, test_ground = dl.load_data(dataset=dataset)

## Synthesis + Verification
Generate functions based on feedback from the verifier (vary cardinality)

In [7]:
from program_synthesis.heuristic_generator import HeuristicGenerator
validation_accuracy = []
training_accuracy = []
validation_coverage = []
training_coverage = []
idx = None

hg = HeuristicGenerator(train_primitive_matrix, val_primitive_matrix, 
                            val_ground, train_ground, 
                            b=0.5)
for i in range(3,6):
    print i
    if i == 3:
        hg.run_synthesizer(max_cardinality=1, idx=idx, keep=3, model='dt')
    else:
        hg.run_synthesizer(max_cardinality=1, idx=idx, keep=1, model='dt')
    hg.run_verifier()
    
    va,ta, vc, tc = hg.evaluate()
    validation_accuracy.append(va)
    training_accuracy.append(ta)
    validation_coverage.append(vc)
    training_coverage.append(tc)
    
    #No feedback needed for this small dataset (none of the data points are vague)
    #hg.find_feedback()
    #idx = hg.feedback_idx

3
4
5


In [8]:
#print "Program Synthesis Train Accuracy: ", np.max(training_accuracy[1:])
print "Program Synthesis Validation Accuracy: ", np.max(validation_accuracy[1:])
print "Program Synthesis Training Set Coverage: ", np.max(training_coverage[1:])

Program Synthesis Validation Accuracy:  0.901785714286
Program Synthesis Training Set Coverage:  1.0


## Calculate Baselines

[**Decision Tree**](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)
Depth bound is unbounded, using default settings. Could do a finer comparison if needed.

In [ ]:
from baselines.models import DecisionTree
dt = DecisionTree(train_primitive_matrix, val_primitive_matrix, 
                            val_ground, train_ground)
dt.fit()
va,ta, vc, tc = dt.evaluate()
print "Decision Tree Validation Accuracy: ", va
print "Training Set Coverage: ", tc

[**Boosting (AdaBoost)**](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier) No bound of the number of estimators yet, could bound by number of primitives for "fair" comparison?

In [ ]:
from baselines.models import BoostClassifier
bc = BoostClassifier(train_primitive_matrix, val_primitive_matrix, 
                            val_ground, train_ground)
bc.fit()
va,ta, vc, tc = bc.evaluate()
print "Boosting Validation Accuracy: ", va
print "Training Set Coverage: ", tc

[**LabelPropagation (Normal Semi-Supervised)**](http://scikit-learn.org/stable/modules/generated/sklearn.semi_supervised.LabelSpreading.html#sklearn.semi_supervised.LabelSpreading)
Semi-supervised method that is robust to noise. Fits with both the labeled and unlabeled examples

In [ ]:
from baselines.models import SemiSupervised
ss = SemiSupervised(train_primitive_matrix, val_primitive_matrix, 
                            val_ground, train_ground)
ss.fit()
va,ta, vc, tc = ss.evaluate()
print "SemiSupervised Validation Accuracy: ", va
print "Training Set Coverage: ", tc